# 🎮 체스 AI 강화학습 (AlphaZero 스타일 MCTS)

**Pre-trained CNN 모델을 MCTS 기반 AlphaZero 스타일로 강화학습합니다.**

## 핵심 개선사항
- **MCTS 탐색**: 시뮬레이션 400회 이상으로 Policy보다 더 나은 수 탐색
- **학습 타겟**: MCTS 방문 횟수 분포를 정책 타겟으로 사용
- **Opponent Pool**: SL 모델을 포함한 상대 풀과 대결
- **게이팅**: 55% 이상 승률 시에만 모델 갱신

## 1. 환경 설정

In [1]:
import os
import copy
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingLR
import numpy as np
from datetime import datetime

# 프로젝트 모듈
from chess_model import ChessCNN, load_model
from train_utils import (
    train_step_mcts, evaluate_vs_opponent, check_weight_diff,
    compute_kl_divergence, compute_masked_entropy,
    create_tensorboard_writer
)
from mcts import (
    MCTS, OpponentPool,
    play_games_batch_with_mcts, play_games_vs_opponent_pool
)
import fast_chess as fc

# 디바이스 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🖥️ 디바이스: {device}")

# CUDA 메모리 최적화
if torch.cuda.is_available():
    torch.backends.cudnn.benchmark = True
    print(f"📊 GPU: {torch.cuda.get_device_name(0)}")
    print(f"💾 VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

🖥️ 디바이스: cuda
📊 GPU: NVIDIA GeForce RTX 5060 Ti
💾 VRAM: 17.1 GB


## 2. 하이퍼파라미터 설정

In [2]:
# =============================================================================
# 학습 설정
# =============================================================================
LEARNING_RATE = 5e-5          # Fine-tuning용 낮은 학습률 (더 보수적)
LEARNING_RATE_MIN = 1e-6      # 최소 학습률
NUM_ITERATIONS = 10000        # 총 학습 반복 횟수
GAMES_PER_ITERATION = 8       # 반복당 Self-play 게임 수 (MCTS는 느리므로 줄임)
MAX_MOVES = 200               # 게임당 최대 수

# =============================================================================
# MCTS 설정 (핵심!)
# =============================================================================
MCTS_SIMULATIONS = 400        # MCTS 시뮬레이션 횟수 (최소 400 권장)
MCTS_C_PUCT = 1.5             # UCB 탐색 상수
MCTS_DIRICHLET_ALPHA = 0.3    # 루트 노이즈 파라미터
MCTS_TEMPERATURE = 1.0        # 초기 탐색 온도
MCTS_TEMP_THRESHOLD = 30      # 이 수 이후 greedy 선택

# =============================================================================
# Opponent Pool 설정
# =============================================================================
OPPONENT_POOL_SIZE = 5        # 최대 상대 모델 수 (SL 모델 제외)
SL_MODEL_WEIGHT = 0.3         # SL 모델 선택 확률 가중치

# =============================================================================
# 손실 함수 가중치
# =============================================================================
VALUE_LOSS_WEIGHT = 1.0       # Value Loss 가중치 (AlphaZero 스타일)
ENTROPY_BONUS = 0.01          # 엔트로피 보너스 (탐색 장려)
KL_PENALTY = 0.1              # KL Divergence 패널티 초기값

# =============================================================================
# KL Divergence 설정 (Adaptive)
# =============================================================================
USE_ADAPTIVE_KL = True        # Adaptive KL 패널티 사용
KL_TARGET = 0.001             # 목표 KL 값
KL_PENALTY_MIN = 1e-4         # 최소 β 값
KL_PENALTY_MAX = 10.0         # 최대 β 값

# =============================================================================
# 평가 및 저장 (게이팅 강화!)
# =============================================================================
EVAL_INTERVAL = 10            # 평가 주기 (iteration)
EVAL_GAMES = 50               # 평가용 대결 게임 수 (50판으로 증가 - 통계적 유의성)
WIN_RATE_THRESHOLD = 0.55     # 저장 기준 승률 (55%+ 로 상향)
SAVE_INTERVAL = 50            # 체크포인트 저장 주기

# =============================================================================
# 경로 설정
# =============================================================================
PRETRAINED_PATH = 'models/best_chess_cnn.pth'
CHECKPOINT_PATH = 'models/chess_cnn_rl_mcts.pth'
BEST_MODEL_PATH = 'models/best_chess_cnn_rl_mcts.pth'

print("✅ 하이퍼파라미터 설정 완료")
print(f"   - 학습률: {LEARNING_RATE} (min: {LEARNING_RATE_MIN})")
print(f"   - 반복 횟수: {NUM_ITERATIONS}")
print(f"   - 게임/반복: {GAMES_PER_ITERATION}")
print(f"   - MCTS 시뮬레이션: {MCTS_SIMULATIONS}회")
print(f"   - 상대 풀 크기: {OPPONENT_POOL_SIZE} + SL 모델")
print(f"   - 게이팅 임계값: {WIN_RATE_THRESHOLD:.0%}")

✅ 하이퍼파라미터 설정 완료
   - 학습률: 5e-05 (min: 1e-06)
   - 반복 횟수: 10000
   - 게임/반복: 8
   - MCTS 시뮬레이션: 400회
   - 상대 풀 크기: 5 + SL 모델
   - 게이팅 임계값: 55%


## 3. 모델 로드

Pre-trained 모델을 로드하고, Reference 모델(고정)과 학습 모델을 준비합니다.

In [3]:
# Pre-trained 모델 로드
print("📥 Pre-trained 모델 로드 중...")

if os.path.exists(PRETRAINED_PATH):
    model, checkpoint = load_model(PRETRAINED_PATH, device)
    print(f"   ✅ {PRETRAINED_PATH} 로드 완료")
else:
    print(f"   ⚠️ {PRETRAINED_PATH} 없음 - 새 모델 생성")
    model = ChessCNN(num_channels=256).to(device)

# 학습 모델로 설정
model.train()

# Reference 모델 (Pre-trained 상태 고정) - KL 패널티 계산용
print("🔒 Reference 모델 생성 중...")
ref_model = ChessCNN(num_channels=256).to(device)
ref_model.load_state_dict(model.state_dict())
ref_model.eval()

for param in ref_model.parameters():
    param.requires_grad = False
print("   ✅ Reference 모델 동결 완료")

# SL 모델 (Opponent Pool용, 고정)
print("📦 SL 모델 생성 중 (Opponent Pool)...")
sl_model = ChessCNN(num_channels=256).to(device)
sl_model.load_state_dict(model.state_dict())
sl_model.eval()

for param in sl_model.parameters():
    param.requires_grad = False
print("   ✅ SL 모델 동결 완료")

# Opponent Pool 생성
print("🎯 Opponent Pool 생성 중...")
opponent_pool = OpponentPool(
    sl_model=sl_model,
    device=device,
    max_opponents=OPPONENT_POOL_SIZE,
    sl_model_weight=SL_MODEL_WEIGHT,
)
print(f"   ✅ Opponent Pool 생성 완료 (SL 모델 포함)")

# Best 모델 (평가용 상대)
best_model = ChessCNN(num_channels=256).to(device)
best_model.load_state_dict(model.state_dict())
best_model.eval()
print("   ✅ Best 모델 초기화 완료")

# 모델 파라미터 수
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"\n📊 모델 파라미터:")
print(f"   - 전체: {total_params:,}")
print(f"   - 학습 가능: {trainable_params:,}")

   ✅ models/best_chess_cnn.pth 로드 완료
🔒 Reference 모델 생성 중...
   ✅ Reference 모델 동결 완료
📦 SL 모델 생성 중 (Opponent Pool)...
   ✅ SL 모델 동결 완료
🎯 Opponent Pool 생성 중...
   ✅ Opponent Pool 생성 완료 (SL 모델 포함)
   ✅ Best 모델 초기화 완료

📊 모델 파라미터:
   - 전체: 9,314,433
   - 학습 가능: 9,314,433


## 4. 옵티마이저 및 TensorBoard 설정

In [4]:
# 옵티마이저
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

# 학습률 스케줄러 (Cosine Annealing)
scheduler = CosineAnnealingLR(
    optimizer, 
    T_max=NUM_ITERATIONS,
    eta_min=LEARNING_RATE_MIN
)
print(f"📉 학습률 스케줄러: CosineAnnealing (max={LEARNING_RATE}, min={LEARNING_RATE_MIN})")

# AMP (Automatic Mixed Precision)
use_amp = torch.cuda.is_available()
scaler = torch.amp.GradScaler('cuda') if use_amp else None
print(f"⚡ AMP: {'활성화' if use_amp else '비활성화'}")

# TensorBoard Writer 생성 (타임스탬프 기반)
writer = create_tensorboard_writer("rl_mcts")
print(f"📊 TensorBoard: {writer.log_dir}")

# 체크포인트에서 이어서 학습
start_iteration = 0
best_win_rate = 0.0

if os.path.exists(CHECKPOINT_PATH):
    ckpt = torch.load(CHECKPOINT_PATH, map_location=device, weights_only=False)
    if 'model_state_dict' in ckpt:
        model.load_state_dict(ckpt['model_state_dict'])
        if 'optimizer_state_dict' in ckpt:
            optimizer.load_state_dict(ckpt['optimizer_state_dict'])
        if 'scheduler_state_dict' in ckpt:
            scheduler.load_state_dict(ckpt['scheduler_state_dict'])
        start_iteration = ckpt.get('iteration', 0)
        best_win_rate = ckpt.get('best_win_rate', 0.0)
        KL_PENALTY = ckpt.get('kl_penalty', KL_PENALTY)
        print(f"✅ 체크포인트 로드: iteration {start_iteration}, win_rate {best_win_rate:.2%}")

print(f"\n🚀 학습 시작 준비 완료 (iteration {start_iteration}부터)")

📉 학습률 스케줄러: CosineAnnealing (max=5e-05, min=1e-06)
⚡ AMP: 활성화
📊 TensorBoard: models/tensorboard/rl_mcts_20260205_114533

🚀 학습 시작 준비 완료 (iteration 0부터)


## 5. 학습 루프

Self-play로 게임을 진행하고 A2C 알고리즘으로 학습합니다.

In [ ]:
from tqdm.auto import tqdm

print("🎮 MCTS 기반 강화학습 시작!")
print("=" * 60)
print(f"   MCTS 시뮬레이션: {MCTS_SIMULATIONS}회")
print(f"   게이팅 임계값: {WIN_RATE_THRESHOLD:.0%}")
print(f"   Opponent Pool: {len(opponent_pool)} 모델")
print("=" * 60)

# 현재 KL Penalty (Adaptive용)
current_kl_penalty = KL_PENALTY

for iteration in tqdm(range(start_iteration, NUM_ITERATIONS), desc="학습 진행"):
    # =========================================================================
    # MCTS Self-play 게임 진행 (Opponent Pool 사용)
    # =========================================================================
    trajectories, results, game_stats = play_games_vs_opponent_pool(
        current_model=model,
        opponent_pool=opponent_pool,
        model_class=ChessCNN,
        device=device,
        num_games=GAMES_PER_ITERATION,
        num_simulations=MCTS_SIMULATIONS,
        c_puct=MCTS_C_PUCT,
        temperature=MCTS_TEMPERATURE,
        temperature_threshold=MCTS_TEMP_THRESHOLD,
        max_moves=MAX_MOVES,
    )
    
    # =========================================================================
    # AlphaZero 스타일 학습 스텝 (MCTS 정책 타겟)
    # =========================================================================
    train_metrics = train_step_mcts(
        model=model,
        ref_model=ref_model,
        optimizer=optimizer,
        trajectories=trajectories,
        results=results,
        device=device,
        value_loss_weight=VALUE_LOSS_WEIGHT,
        entropy_bonus=ENTROPY_BONUS,
        kl_penalty=current_kl_penalty,
        max_grad_norm=1.0,
        scaler=scaler,
        use_amp=use_amp
    )
    
    # =========================================================================
    # 학습률 스케줄러 업데이트
    # =========================================================================
    scheduler.step()
    current_lr = scheduler.get_last_lr()[0]
    
    # =========================================================================
    # Adaptive KL Penalty
    # =========================================================================
    if USE_ADAPTIVE_KL:
        avg_kl = train_metrics['kl_div']
        if avg_kl > KL_TARGET * 1.5:
            current_kl_penalty = min(current_kl_penalty * 1.5, KL_PENALTY_MAX)
        elif avg_kl < KL_TARGET * 0.5:
            current_kl_penalty = max(current_kl_penalty / 1.5, KL_PENALTY_MIN)
    
    # =========================================================================
    # TensorBoard 로깅
    # =========================================================================
    writer.add_scalar('Loss/Total', train_metrics['total_loss'], iteration)
    writer.add_scalar('Loss/Policy', train_metrics['policy_loss'], iteration)
    writer.add_scalar('Loss/Value', train_metrics['value_loss'], iteration)
    writer.add_scalar('Entropy', train_metrics['entropy'], iteration)
    writer.add_scalar('KL_Divergence', train_metrics['kl_div'], iteration)
    writer.add_scalar('KL_Penalty', current_kl_penalty, iteration)
    writer.add_scalar('Learning_Rate', current_lr, iteration)
    writer.add_scalar('Games/Avg_Moves', game_stats['avg_moves'], iteration)
    writer.add_scalar('Games/Wins', game_stats['wins'], iteration)
    writer.add_scalar('Games/Losses', game_stats['losses'], iteration)
    writer.add_scalar('Games/Draws', game_stats['draws'], iteration)
    writer.add_scalar('Games/Win_Rate', game_stats['win_rate'], iteration)
    writer.add_scalar('Opponent_Pool_Size', len(opponent_pool), iteration)
    
    # =========================================================================
    # 평가 및 게이팅 (주기적)
    # =========================================================================
    if (iteration + 1) % EVAL_INTERVAL == 0:
        # SL 모델과 대결 (기준점)
        win_rate_vs_sl = evaluate_vs_opponent(
            current_model=model,
            opponent_model=sl_model,
            device=device,
            num_games=EVAL_GAMES,
            temperature=0.3
        )
        
        # Best 모델과 대결
        win_rate_vs_best = evaluate_vs_opponent(
            current_model=model,
            opponent_model=best_model,
            device=device,
            num_games=EVAL_GAMES,
            temperature=0.3
        )
        
        writer.add_scalar('Eval/Win_Rate_vs_SL', win_rate_vs_sl, iteration)
        writer.add_scalar('Eval/Win_Rate_vs_Best', win_rate_vs_best, iteration)
        
        # 게이팅: 55% 이상 승률 시에만 모델 갱신
        if win_rate_vs_best >= WIN_RATE_THRESHOLD:
            # Opponent Pool에 추가
            opponent_pool.add_opponent(model, win_rate_vs_best)
            
            # Best 모델 갱신
            if win_rate_vs_best > best_win_rate:
                best_win_rate = win_rate_vs_best
                best_model.load_state_dict(model.state_dict())
                
                # Best 모델 저장
                torch.save({
                    'model_state_dict': model.state_dict(),
                    'iteration': iteration,
                    'win_rate': win_rate_vs_best,
                    'kl_penalty': current_kl_penalty,
                }, BEST_MODEL_PATH)
                
                tqdm.write(f"🏆 [Iter {iteration+1}] 새로운 최고 모델! "
                          f"vs Best: {win_rate_vs_best:.2%}, vs SL: {win_rate_vs_sl:.2%}")
            else:
                tqdm.write(f"📈 [Iter {iteration+1}] Opponent Pool에 추가! "
                          f"vs Best: {win_rate_vs_best:.2%}")
        else:
            tqdm.write(f"📊 [Iter {iteration+1}] 게이팅 실패 - "
                      f"vs Best: {win_rate_vs_best:.2%} (필요: {WIN_RATE_THRESHOLD:.0%}), "
                      f"vs SL: {win_rate_vs_sl:.2%}")
    
    # =========================================================================
    # 체크포인트 저장 (주기적)
    # =========================================================================
    if (iteration + 1) % SAVE_INTERVAL == 0:
        torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            'iteration': iteration + 1,
            'best_win_rate': best_win_rate,
            'kl_penalty': current_kl_penalty,
        }, CHECKPOINT_PATH)
        tqdm.write(f"💾 [Iter {iteration+1}] 체크포인트 저장")
    
    # =========================================================================
    # 진행 상황 출력
    # =========================================================================
    if (iteration + 1) % 10 == 0:
        weight_diff = check_weight_diff(model, ref_model)
        tqdm.write(f"📈 [Iter {iteration+1}] Loss: {train_metrics['total_loss']:.4f}, "
                   f"LR: {current_lr:.2e}, "
                   f"KL: {train_metrics['kl_div']:.6f}, "
                   f"Moves: {game_stats['avg_moves']:.1f}, "
                   f"Pool: {len(opponent_pool)}")

print("\n" + "=" * 60)
print("✅ 학습 완료!")

# 최종 저장
torch.save({
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'scheduler_state_dict': scheduler.state_dict(),
    'iteration': NUM_ITERATIONS,
    'best_win_rate': best_win_rate,
    'kl_penalty': current_kl_penalty,
}, CHECKPOINT_PATH)
print(f"💾 최종 체크포인트 저장: {CHECKPOINT_PATH}")

writer.close()

학습 진행:   0%|          | 0/10000 [00:00<?, ?it/s]

## 6. 학습 결과 분석

## 7. 샘플 게임 시각화

학습된 모델로 샘플 게임을 진행합니다.

In [ ]:
def play_demo_game_mcts(model, device, num_simulations=100, max_moves=100, verbose=True):
    """MCTS를 사용하여 샘플 게임을 진행하고 결과를 출력합니다."""
    state = fc.create_initial_state()
    model.eval()
    
    mcts = MCTS(
        model=model,
        device=device,
        num_simulations=num_simulations,
        c_puct=1.5,
        temperature=0.0,  # Greedy for demo
    )
    
    moves_history = []
    
    for move_num in range(max_moves):
        if fc.is_game_over(state):
            break
        
        mask_tensor = torch.from_numpy(fc.legal_move_mask_fast(state))
        
        if mask_tensor.sum() == 0:
            break
        
        # MCTS 탐색
        action, mcts_probs, value = mcts.select_action(state, add_noise=False)
        
        # 수 변환
        from_sq = action // 64
        to_sq = action % 64
        files = "abcdefgh"
        ranks = "12345678"
        move_str = f"{files[from_sq & 7]}{ranks[from_sq >> 3]}{files[to_sq & 7]}{ranks[to_sq >> 3]}"
        moves_history.append(move_str)
        
        fc.make_move(state, action)
    
    # 결과
    result = fc.get_result(state)
    if result == 1.0:
        result_str = "1-0 (백 승)"
    elif result == 0.0:
        result_str = "0-1 (흑 승)"
    else:
        result_str = "1/2-1/2 (무승부)"
    
    if verbose:
        print(f"🎮 샘플 게임 결과: {result_str}")
        print(f"   총 {len(moves_history)}수")
        print(f"   수순: {' '.join(moves_history[:20])}{'...' if len(moves_history) > 20 else ''}")
    
    return moves_history, result_str

# 샘플 게임 3개 진행 (MCTS 사용)
print("🎲 MCTS 샘플 게임 진행 중...")
print("=" * 60)

for i in range(3):
    print(f"\n게임 #{i+1}")
    play_demo_game_mcts(model, device, num_simulations=100)